In [9]:
import requests

url = 'https://ultra-jinja---warmup.challenge.haruulzangi.mn/'
data = {'username': 'Hello mr ninja jinja'}
response = requests.post(url, data=data)

print(response.text)

Hello, Hello mr ninja jinja!


In [ ]:
import pickle, base64

index_to_check = 

payload = b'\x80\x04'  # PROTO 4
payload += b'('        # MARK
payload += b']'        # EMPTY_LIST
payload += b'\x94'     # MEMOIZE index 0

payload += b'h' + bytes([index_to_check])  # BINGET 49
payload += b'e'  # APPENDS
payload += b'.'  # STOP

b64_payload = base64.b64encode(payload).decode()
print(b64_payload)

gAQoXZRo/2Uu


In [57]:
payload

b'\x80\x04(]\x94h-e.'

In [29]:
pickled_payload

b'cnt\nsystem\np0\n(Va\np1\ntp2\nRp3\n.'

Recovered key bytes: [97]
Decrypted flag: z8b35B	iP{2@o/8,Mi
g_S(A


'ajf\x01C\x19cHN9r\x12pxw+\x00\nsI;\x14TuCW6\x12eq\x1c$(S:'

In [55]:
''.join([chr(x ^ 0x1A) for x in data])

'a§jìf\x01C\x19c\x86HïÑ\x94Nü\x9eí9rü\x12pxw+Â\x00\nsÛI;\x14TuCW6\x12ùeçqÒ\x1c$(SË:\x8a'

In [61]:
def solve():
    """
    Re-implements the C code's logic to generate the secret string.
    """
    
    # --- YOU MUST EXTRACT THESE VALUES FROM THE BINARY ---
    # These are placeholder values to make the script runnable.
    # Use a disassembler or debugger to find the actual data.
    
    # g_404010: The length of the flag.
    FLAG_LENGTH = 29
    
    # g_4021f8: The first data array (bytes).
    DATA_ARRAY_1 = [
        0x2e, 0x46, 0x61, 0x69, 0x74, 0x68, 0x20, 0x69, 0x73, 0x20, 
        0x74, 0x68, 0x65, 0x20, 0x62, 0x69, 0x72, 0x64, 0x20, 0x74, 
        0x68, 0x61, 0x74, 0x20, 0x66, 0x65, 0x65, 0x6c, 0x73
    ]
    
    # The integer array at 0x4040C0 (4202944).
    DATA_ARRAY_2 = [
        0x5a, 0x07, 0x09, 0x1a, 0x11, 0x18, 0x4f, 0x1a, 0x17, 0x4f, 
        0x01, 0x18, 0x1b, 0x4f, 0x1c, 0x1e, 0x01, 0x00, 0x4f, 0x01, 
        0x18, 0x1f, 0x01, 0x4f, 0x1e, 0x1b, 0x1b, 0x1c, 0x07
    ]
    # ----------------------------------------------------

    flag = bytearray()

    # Initialize v2 and v3 for the first character (i=0)
    v2 = 90
    v3 = 46

    for i in range(FLAG_LENGTH):
        # On the first iteration, use the initial v2 and v3
        if i > 0:
            # For all other iterations, update v2 and v3 from the data arrays
            v3 = DATA_ARRAY_1[i]
            v2 = DATA_ARRAY_2[i]

        # Generate the next character of the flag
        flag_char = (v3 ^ v2) & 0xFF
        flag.append(flag_char)
        
    print("[+] Successfully generated the sacred word! ✨")
    print(f"FLAG: {flag.decode('utf-8')}")


if __name__ == '__main__':
    solve()

[+] Successfully generated the sacred word! ✨
FLAG: tAhseposdoup~o~wsdoup~uox~~pt


In [65]:
def reverse_encryption(encrypted_text, key):
    """
    Accurately reverses the encryption logic from the provided C code.

    The function iterates through each character of the encrypted text and
    applies the correct decryption formula based on its type (uppercase,
    lowercase, digit, or symbol) using a repeating key.

    Args:
        encrypted_text (str): The string to be decrypted.
        key (list[int]): The integer key array used for decryption.

    Returns:
        str: The decrypted plaintext string.
    """
    decrypted_chars = []
    key_length = len(key)

    for i, char in enumerate(encrypted_text):
        # Select the current key value based on character index
        current_key = key[i % key_length]
        char_code = ord(char)

        if 'A' <= char <= 'Z':
            # Reverse the logic for uppercase letters:
            # C equivalent: ((char_code - 'A' - key + 26) % 26) + 'A'
            new_char_code = ((char_code - ord('A') - current_key + 26) % 26) + ord('A')
            decrypted_chars.append(chr(new_char_code))
        
        elif 'a' <= char <= 'z':
            # Reverse the logic for lowercase letters:
            # C equivalent: ((char_code - 'a' - key + 26) % 26) + 'a'
            new_char_code = ((char_code - ord('a') - current_key + 26) % 26) + ord('a')
            decrypted_chars.append(chr(new_char_code))

        elif '0' <= char <= '9':
            # Reverse the logic for digits:
            # C equivalent: ((char_code - '0' - key + 10) % 10) + '0'
            print(current_key)
            new_char_code = ((char_code - ord('0') - current_key + 10) % 10) + ord('0')
            decrypted_chars.append(chr(new_char_code))

        else:
            # Any other characters (symbols) are not changed
            decrypted_chars.append(char)
            
    return "".join(decrypted_chars)

def find_key(ciphertext, plaintext):
    """
    Derives the encryption key using a known-plaintext attack by comparing
    a ciphertext segment with its corresponding plaintext.

    Args:
        ciphertext (str): A piece of the encrypted text (e.g., "KM20").
        plaintext (str): The corresponding known decrypted text (e.g., "HZ20").

    Returns:
        list[int]: The derived key as a list of integers.
    """
    if len(ciphertext) != len(plaintext):
        raise ValueError("Ciphertext and plaintext must have the same length.")

    key = []
    for c_char, p_char in zip(ciphertext, plaintext):
        c_code = ord(c_char)
        p_code = ord(p_char)

        if 'A' <= c_char <= 'Z' and 'A' <= p_char <= 'Z':
            # key = (ciphertext_char - plaintext_char + 26) % 26
            k = (c_code - p_code + 26) % 26
            key.append(k)
        elif 'a' <= c_char <= 'z' and 'a' <= p_char <= 'z':
            # key = (ciphertext_char - plaintext_char + 26) % 26
            k = (c_code - p_code + 26) % 26
            key.append(k)
        elif '0' <= c_char <= '9' and '0' <= p_char <= '9':
            # key = (ciphertext_char - plaintext_char + 10) % 10
            k = (c_code - p_code + 10) % 10
            key.append(k)
        elif c_char == p_char:
            # If characters are identical (like symbols), the key value is 0.
            key.append(0)
        else:
            raise ValueError(f"Cannot determine key for mismatched types: '{c_char}' and '{p_char}'")
            
    return key

# --- Main execution block ---

# The original encrypted message from the binary
encrypted_string = "KM2025{7iyx_y9kdtyk_jo$uu}"

# Known parts of the plaintext and ciphertext to derive the key
known_ciphertext_prefix = "KM20"
known_plaintext_prefix = "HZ20"

print(f"[*] Deriving key using known plaintext attack...")
print(f"    Ciphertext: {known_ciphertext_prefix}")
print(f"    Plaintext:  {known_plaintext_prefix}")

# Automatically find the key based on the known prefixes
derived_key = find_key(known_ciphertext_prefix, known_plaintext_prefix)

# Perform the full decryption using the auto-derived key
decrypted_string = reverse_encryption(encrypted_string, derived_key)

# The expected flag format is HZ2025{...}, so we format our result accordingly
final_flag = f"HZ2025{{{decrypted_string[7:]}"

# 🚩 Print the analysis and the final, correct flag
print("\n[*] Decrypting the full string...")
print(f"Encrypted String: {encrypted_string}")
print(f"Derived Key:      {derived_key}")
print(f"Full Decryption:  {decrypted_string}")
print("-" * 40)
print(f"Final Flag:       {final_flag}")



[*] Deriving key using known plaintext attack...
    Ciphertext: KM20
    Plaintext:  HZ20
0
0
3
13
0
13

[*] Decrypting the full string...
Encrypted String: KM2025{7iyx_y9kdtyk_jo$uu}
Derived Key:      [3, 13, 0, 0]
Full Decryption:  HZ2092{7flx_v6kdqlk_gb$ur}
----------------------------------------
Final Flag:       HZ2025{7flx_v6kdqlk_gb$ur}
